In [2]:
"""
1) Query DB:
SELECT 
    OBJECT_NAME(f.parent_object_id) AS TableName,
    OBJECT_NAME (f.referenced_object_id) AS ReferenceTableName
FROM 
    sys.foreign_keys AS f
INNER JOIN 
    sys.foreign_key_columns AS fc 
ON 
    f.OBJECT_ID = fc.constraint_object_id

2) ChatGPT:
Converti la seguente tabella in una serie di tuple.
Per esempio: 
("RoleRequestApproval", "FK_RoleRequestApproval_User"),
("RoleRequestApproval", "FK_RoleRequestApproval_User_CompletedBy"),

.................................................

3) inserisci le tuple nel codice ed esegui
"""

import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, Output, HBox, Checkbox

# Inserisci qui le tue relazioni
relazioni = [
    ("ApplicationRolePermission", "ApplicationRole"),
    ("ApplicationRoleUser", "ApplicationRole"),
    ("RoleRequest", "ApplicationRole"),
    ("ApplicationRoleApprover", "ApplicationRole"),
    ("Device", "DeviceFamily"),
    ("DeviceRole_User", "DeviceRole"),
    ("PhysicalDevice_User", "DeviceRole"),
    ("DeviceRolePermission", "DeviceRole"),
    ("ObjectId", "ObjectIdsRequest"),
    ("PhysicalDevice_User", "PhysicalDevice"),
    ("Device_DeviceCategory", "DeviceCategory"),
    ("RoleRequestApproval", "RoleRequest"),
    ("DeviceRole", "Device"),
    ("PhysicalDevice", "Device"),
    ("Device_DeviceCategory", "Device"),
    ("DevicePermission", "Device"),
    ("ApplicationPropertyVal", "ApplicationRoleUser"),
    ("File", "FileType"),
    ("NotificationTarget", "Notification"),
    ("FeatureAccess", "Feature"),
    ("DeviceRolePermission", "DevicePermission"),
    ("Notification", "NotificationTopic"),
    ("ReportDownload", "Report"),
    ("ApplicationPropertyVal", "ApplicationPermissionUser"),
    ("ApplicationPropertyVal", "ApplicationRolePermission"),
    ("ApplicationPropertyVal", "ApplicationPropertyDef"),
    ("ApplicationPropertyVal", "ApplicationPropertyDef"),
    ("FileSharing", "File"),
    ("FileVersion", "File"),
    ("UserGroup", "Group"),
    ("FileSharing", "Group"),
    ("RoleRequestApproval", "User"),
    ("RoleRequestApproval", "User"),
    ("ApplicationRoleApprover", "User"),
    ("PhysicalDevice", "User"),
    ("PhysicalDevice_User", "User"),
    ("RoleRequest", "User"),
    ("RoleRequest", "User"),
    ("DeviceRole_User", "User"),
    ("File", "User"),
    ("Relt", "User"),
    ("Relt", "User"),
    ("FileSharing", "User"),
    ("FileVersion", "User"),
    ("AccessRequest", "User"),
    ("FeatureAccess", "User"),
    ("ApplicationPermissionUser", "User"),
    ("ReportDownload", "User"),
    ("ApplicationRoleUser", "User"),
    ("Group", "User"),
    ("UserGroup", "User"),
    ("ApplicationPermissionUser", "ApplicationPermission"),
    ("ApplicationRolePermission", "ApplicationPermission"),
    ("ApplicationRole", "Application"),
    ("FeatureAccess", "Application"),
    ("Notification", "Application"),
    ("ApplicationPropertyVal", "Application"),
    ("NotificationTopic", "Application"),
    ("ApplicationPermission", "Application"),
    ("ApplicationPropertyDef", "Application"),
    ("User", "Application"),
]

# Crea un grafo vuoto
G = nx.DiGraph()  # Usa DiGraph per un grafo diretto

# Aggiungi i nodi e gli archi al grafo
for relazione in relazioni:
    G.add_edge(*relazione)

# Crea una disposizione per i nodi del grafo
pos = nx.spring_layout(G, k=0.5, iterations=20)
# pos = nx.shell_layout(G)
# pos = nx.spectral_layout(G)
# pos = nx.circular_layout(G)

# Crea un elenco di tutti i nodi
all_nodes = sorted(list(G.nodes()))

def draw_graph(nodes_to_show=all_nodes, show_incoming_edges=False, show_labels=True):
    # Se nodes_to_show è una stringa vuota, mostrare tutti i nodi, altrimenti filtrare
    if nodes_to_show == '':
        nodes = all_nodes
    else:
        nodes = []
        for node in nodes_to_show:
            ego_graph = nx.ego_graph(G if show_incoming_edges else nx.Graph(G), node)
            nodes.extend(ego_graph.nodes())
        nodes = list(set(nodes))  # rimuove i duplicati


    # Crea una mappa di colori per i nodi
    color_map = {node: plt.cm.Pastel2(i / len(nodes)) for i, node in enumerate(nodes)}

    # Grandezza grafico
    plt.rcParams["figure.figsize"] = [23, 9]

    # Disegna i nodi con colori diversi
    nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=[color_map[node] for node in nodes])

    # Disegna gli archi con lo stesso colore del nodo di partenza
    for n1, n2 in G.edges():
        if n1 in nodes and n2 in nodes:
            nx.draw_networkx_edges(G, pos, edgelist=[(n1, n2)], edge_color=color_map[n1])

    # Disegna le etichette dei nodi se show_labels è True
    if show_labels:
        nx.draw_networkx_labels(G, pos, labels={node: node for node in nodes}, font_size=10)

    plt.show()

# Aggiungi un widget Checkbox e SelectMultiple
show_labels_check = Checkbox(value=True, description='Mostra etichette')
show_incoming_edges_check = Checkbox(value=False, description='NO collegamenti entrata')
show_nodes_SelectMultiple = SelectMultiple(options=all_nodes, value=all_nodes, rows=len(all_nodes),description='Nodi:')

# Usa la funzione interact per creare controlli interattivi
interact(draw_graph, nodes_to_show=show_nodes_SelectMultiple, show_incoming_edges=show_incoming_edges_check, show_labels=show_labels_check)

interactive(children=(SelectMultiple(description='Nodi:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

<function __main__.draw_graph(nodes_to_show=['AccessRequest', 'Application', 'ApplicationPermission', 'ApplicationPermissionUser', 'ApplicationPropertyDef', 'ApplicationPropertyVal', 'ApplicationRole', 'ApplicationRoleApprover', 'ApplicationRolePermission', 'ApplicationRoleUser', 'Device', 'DeviceCategory', 'DeviceFamily', 'DevicePermission', 'DeviceRole', 'DeviceRolePermission', 'DeviceRole_User', 'Device_DeviceCategory', 'Feature', 'FeatureAccess', 'File', 'FileSharing', 'FileType', 'FileVersion', 'Group', 'Notification', 'NotificationTarget', 'NotificationTopic', 'ObjectId', 'ObjectIdsRequest', 'PhysicalDevice', 'PhysicalDevice_User', 'Relt', 'Report', 'ReportDownload', 'RoleRequest', 'RoleRequestApproval', 'User', 'UserGroup'], show_incoming_edges=False, show_labels=True)>